# 7. Data Manipulation V - Working With Data Types - Quiz

## 7.0. Import `polars` and Load Data

In [1]:
import polars as pl

In [2]:
zone_column_rename_mapping = {
    "LocationID": "location_id",
    "Borough": "borough",
    "Zone": "zone",
}
zones_df = (
    pl.read_parquet("../data/taxi_zone_lookup.parquet")
    .rename(zone_column_rename_mapping)
)

In [3]:
yellow_rides_column_rename_mapping = {
    "VendorID": "vendor_id",
    "RatecodeID": "ratecode_id",
    "PULocationID": "pu_location_id",
    "DOLocationID": "do_location_id",
    "Airport_fee": "airport_fee",
}

rides_df = (
    pl.read_parquet("../data/yellow_tripdata_2024-03.parquet")
    .rename(yellow_rides_column_rename_mapping)
    .join(
        zones_df.select(pl.all().name.prefix("pu_")),
        on="pu_location_id",
    )
    .join(
        zones_df.select(pl.all().name.prefix("do_")),
        on="do_location_id",
    )
)

## 7.1 Question 1: Highest Average Fare by Day and Hour

Extract the day of the week (as a string) and the hour from the `tpep_pickup_datetime` column. Then, calculate the average fare amount for each day-hour combination. Finally, sort the results by average fare amount. Which day-hour combination has the highest average fare amount?

In [4]:
result = (
    rides_df
    #### YOUR CODE HERE
)
print(result)

shape: (168, 3)
┌─────────────┬──────┬───────────┐
│ day_of_week ┆ hour ┆ avg_fare  │
│ ---         ┆ ---  ┆ ---       │
│ i8          ┆ i8   ┆ f64       │
╞═════════════╪══════╪═══════════╡
│ 3           ┆ 4    ┆ 28.732735 │
│ 2           ┆ 4    ┆ 28.361275 │
│ 1           ┆ 4    ┆ 28.345536 │
│ 1           ┆ 1    ┆ 28.123612 │
│ 7           ┆ 23   ┆ 27.736774 │
│ …           ┆ …    ┆ …         │
│ 6           ┆ 10   ┆ 15.954722 │
│ 6           ┆ 1    ┆ 15.582336 │
│ 6           ┆ 3    ┆ 15.483906 │
│ 7           ┆ 2    ┆ 15.34928  │
│ 6           ┆ 2    ┆ 14.980229 │
└─────────────┴──────┴───────────┘


1. day=6, hour=2
2. day=3, hour=4
3. day=7, hour=23
4. day=1, hour=0

## 7.2 Question 2: Dropoff Zone with Longest Trip Duration

Which `do_zone` has the highest trip duration (where "trip duration" is measured as the `.total_seconds()` between `tpep_pickup_datetime` and `tpep_dropoff_datetime`)?

In [5]:
result = (
    rides_df
    #### YOUR CODE HERE
)
print(result)

shape: (1, 2)
┌─────────────────────────────────┬───────────────┐
│ do_zone                         ┆ trip_duration │
│ ---                             ┆ ---           │
│ str                             ┆ i64           │
╞═════════════════════════════════╪═══════════════╡
│ Saint Michaels Cemetery/Woodsi… ┆ 545553        │
└─────────────────────────────────┴───────────────┘


1. Saint Michaels Cemetery/Woodside
2. Midtown Center
3. 207
4. Woodside

## 7.3 Question 3: Pickup Location with Most Diverse Dropoff Zones

With a group-by in `polars`, instead of finding some aggregate summary statistic for each group, you can also collect all the elements for each group into a list by simply passing in the column you'd like to aggregate to a list as a name (see below). With this, for each `pu_location_id`, make a column that aggregates all the `do_zones` associated with that `pu_location_id`; what is the `pu_location_id` with the longest list of associated `do_zone`s (hint: use the `.list` namespace)?

In [6]:
result = (
    rides_df
    .group_by("pu_location_id")
    .agg(pl.col("do_zone"))
    #### YOUR CODE HERE
)
print(result)

shape: (259, 2)
┌────────────────┬─────────────────────────────────┐
│ pu_location_id ┆ do_zone                         │
│ ---            ┆ ---                             │
│ i32            ┆ list[str]                       │
╞════════════════╪═════════════════════════════════╡
│ 161            ┆ ["Lenox Hill West", "Greenwich… │
│ 132            ┆ ["South Jamaica", "Clinton Hil… │
│ 237            ┆ ["Lenox Hill West", "Union Sq"… │
│ 236            ┆ ["Upper East Side South", "Blo… │
│ 162            ┆ ["Flatiron", "Midtown Center",… │
│ …              ┆ …                               │
│ 199            ┆ ["Midtown Center", "Times Sq/T… │
│ 84             ┆ ["Sheepshead Bay"]              │
│ 156            ┆ ["Coney Island"]                │
│ 109            ┆ ["Great Kills"]                 │
│ 5              ┆ ["Arden Heights"]               │
└────────────────┴─────────────────────────────────┘


1. 161
2. 5
3. Midtown Center
4. Arden Heights

## 7.4 Question 4: Most Common Second Word in Zone Names

Using just `zones_df`, split `zone` into a list on `" "` as seen during the module, and take the 2nd element of every list using `.list.get()`. Then, using `group_by`, answer the question--what is the most commonly occurring second word in `zones_df` (excluding `null`)?

In [9]:
result = (
    zones_df
    .with_columns(pl.col("zone").str.split(" ").alias("zone_splitted"))
    #### YOUR CODE HERE
)
print(result)

shape: (11, 2)
┌───────────────┬────────────────┐
│ zone_splitted ┆ num_occurences │
│ ---           ┆ ---            │
│ str           ┆ u32            │
╞═══════════════╪════════════════╡
│ null          ┆ 250            │
│ North         ┆ 3              │
│ Island        ┆ 3              │
│ South         ┆ 2              │
│ West          ┆ 1              │
│ …             ┆ …              │
│ Plaza         ┆ 1              │
│ Village       ┆ 1              │
│ Avenue        ┆ 1              │
│ Beach         ┆ 1              │
│ Point         ┆ 1              │
└───────────────┴────────────────┘


1. Park
2. `null`
3. East
4. North

## 7.7 Question 7: Zones Containing 'North'

How many zones in `zones_df` contain the word "North"?

In [33]:
result = (
    zones_df
    #### YOUR CODE HERE
)
print(result)

shape: (1, 1)
┌──────┐
│ zone │
│ ---  │
│ u32  │
╞══════╡
│ 15   │
└──────┘


1. 15
2. 0
3. 265
4. 19

## 7.8 Question 8: Most Common First Word in Zone Names (Reversed)

For `zones_df`, what is the most common `0th` word in the `zone` column, spelled in reverse? (Hint: split the `zone` column by `" "` into a list of strings; then, take the `0th` element of each list in that column, and apply `.str.reverse()` to it.)

In [37]:
result = (
    zones_df
    #### YOUR CODE HERE
)
print(result)

shape: (195, 2)
┌─────────────┬─────────────────┐
│ zone        ┆ num_occurrences │
│ ---         ┆ ---             │
│ str         ┆ u32             │
╞═════════════╪═════════════════╡
│ tsaE        ┆ 13              │
│ tseW        ┆ 5               │
│ nwotdiM     ┆ 4               │
│ htuoS       ┆ 4               │
│ reppU       ┆ 4               │
│ …           ┆ …               │
│ noskcaJ     ┆ 1               │
│ dlroW       ┆ 1               │
│ sdnaltalF   ┆ 1               │
│ hgielretseW ┆ 1               │
│ sserpyC     ┆ 1               │
└─────────────┴─────────────────┘


1. East
2. tseW
3. kraP
4. tsaE